Danielle Paes Barretto de Arruda Camara

**Version: 30/07/19 (last review)**

Define the higher bound after percentaille 0.90 to avoid same songs.




# Importing libraries

In [1]:
import pandas as pd
import numpy as np
import glob

import matplotlib.pyplot as plt
import seaborn as sns

import time
TodaysDate = time.strftime("%Y-%m-%d")

import pickle
import itertools
import random

# Functions

In [2]:
def frange(start, stop, step):
    """ to use range with decimal numbers"""

    i = start
    while i < stop:
        yield round(i,2)
        i += step

# Folders and Files

In [3]:
snippets_address_folder = "./data/snippets_address/"

# to save processed data to be used in the creation of the playlists

creating_playlist_folder = "./data/creating_playlist/"

In [4]:
snippets_pop = "snippets_address_pop_2019-06-13.csv"

tracks_file_clean = "audio_tracks_cleaned_pop_2019-06-13.csv"

diff_all = "audio_tracks_diff_ALL_cleaned_pop_2019-06-13.csv"

diff_in_playlists = "audio_tracks_diff_in_playlists_cleaned_pop_2019-06-13.csv"

category = snippets_pop.split('.')[0].split('_')[-2]
category

'pop'

# Acessing tracks with snippets
Not all tracks have snippets available so we will need to filter also considering the availability of snippets.

In [5]:
df_snippets = pd.read_csv(snippets_address_folder+snippets_pop)
df_snippets.head()

,track_id,track_preview_url
0,2hnxrRNzF74mdDzpQZQukQ,https://p.scdn.co/mp3-preview/ecf605ea149db4f7...
1,2n48BjaFSczfeIG3HaIaPv,https://p.scdn.co/mp3-preview/c6bb499f1f475493...
2,7DHApOHsIFo7EyfieArYGE,https://p.scdn.co/mp3-preview/44e8764f6028b38a...
3,3ciQQMSy1lKDqJbwU9l1LT,https://p.scdn.co/mp3-preview/c6a9592968a0d4d0...
4,2nFAQBrEEHA7nvYgw6lpD0,https://p.scdn.co/mp3-preview/2fd9b59c151003d9...


In [6]:
df_snippets.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1061 entries, 0 to 1060
Data columns (total 2 columns):
track_id             1061 non-null object
track_preview_url    1061 non-null object
dtypes: object(2)
memory usage: 16.7+ KB


In [7]:
tracks_with_snippets = df_snippets.track_id.tolist()

## Creating new df_diff with only track_id that has snippets

In [8]:
df_tracks_diff = pd.read_csv(creating_playlist_folder+diff_all)

In [9]:
current_track_list = [df_tracks_diff['current_track-next_track'][idx].split('_')[0] for idx in range(df_tracks_diff.shape[0])]

next_track_list = [df_tracks_diff['current_track-next_track'][idx].split('_')[1] for idx in range(df_tracks_diff.shape[0])]

In [10]:
df_tracks_diff['current_track'] = current_track_list
df_tracks_diff['next_track'] = next_track_list

In [11]:
df_tracks_diff.head()

,current_track-next_track,diff_acousticness,diff_danceability,diff_energy,diff_valence,diff_tempo,diff_instrumentalness,diff_key,diff_mode,diff_liveness,diff_loudness,diff_speechiness,is_good_sequence,current_track,next_track
0,2hloaUoRonYssMuqLCBLTX_5wo3Opxvcc5SsszgmwV9Lh,0.18350,0.057,0.048,0.088,47.011,0.000157,2.0,0.0,0.047,1.020,0.2096,1,2hloaUoRonYssMuqLCBLTX,5wo3Opxvcc5SsszgmwV9Lh
1,5wo3Opxvcc5SsszgmwV9Lh_1uXbwHHfgsXcUKfSZw5ZJ0,0.26004,0.032,0.186,0.205,22.121,0.000047,0.0,1.0,0.203,1.971,0.1440,1,5wo3Opxvcc5SsszgmwV9Lh,1uXbwHHfgsXcUKfSZw5ZJ0
2,1uXbwHHfgsXcUKfSZw5ZJ0_7sKDftgGirHWTVFMtJoDoh,0.05624,0.096,0.103,0.209,29.110,0.000047,5.0,0.0,0.261,1.191,0.0330,1,1uXbwHHfgsXcUKfSZw5ZJ0,7sKDftgGirHWTVFMtJoDoh
3,7sKDftgGirHWTVFMtJoDoh_3e9HZxeyfWwjeyPAMmWSSQ,0.16780,0.112,0.143,0.139,8.990,0.000000,4.0,0.0,0.010,0.206,0.0442,1,7sKDftgGirHWTVFMtJoDoh,3e9HZxeyfWwjeyPAMmWSSQ
4,3e9HZxeyfWwjeyPAMmWSSQ_44n97yHySt0Z9rqPaXgjCK,0.68400,0.204,0.403,0.061,32.863,0.000000,4.0,1.0,0.016,3.742,0.0291,1,3e9HZxeyfWwjeyPAMmWSSQ,44n97yHySt0Z9rqPaXgjCK


In [12]:
# Keeping only the row with current_track and next_track within tracks_with_snippets

df_diff_with_snippets = df_tracks_diff[df_tracks_diff['current_track'].isin(tracks_with_snippets)]
df_diff_with_snippets = df_diff_with_snippets[df_diff_with_snippets['next_track'].isin(tracks_with_snippets)]
df_diff_with_snippets.head()

,current_track-next_track,diff_acousticness,diff_danceability,diff_energy,diff_valence,diff_tempo,diff_instrumentalness,diff_key,diff_mode,diff_liveness,diff_loudness,diff_speechiness,is_good_sequence,current_track,next_track
62,2nFAQBrEEHA7nvYgw6lpD0_7DHApOHsIFo7EyfieArYGE,0.3701,0.244,0.066,0.135,90.104,0.000000,10.0,1.0,0.0760,0.306,0.0985,1,2nFAQBrEEHA7nvYgw6lpD0,7DHApOHsIFo7EyfieArYGE
63,7DHApOHsIFo7EyfieArYGE_6xUy203RnyyOfbqf96Nven,0.1181,0.245,0.336,0.255,4.895,0.000000,6.0,1.0,0.0530,1.890,0.2375,1,7DHApOHsIFo7EyfieArYGE,6xUy203RnyyOfbqf96Nven
64,6xUy203RnyyOfbqf96Nven_5tAa8Uaqr4VvA3693mbIhU,0.2530,0.258,0.090,0.102,31.963,0.000000,4.0,1.0,0.0291,1.036,0.2242,1,6xUy203RnyyOfbqf96Nven,5tAa8Uaqr4VvA3693mbIhU
65,5tAa8Uaqr4VvA3693mbIhU_5N4erncE7kuUccm7zEmwzk,0.4126,0.077,0.029,0.080,6.737,0.000003,11.0,0.0,0.2411,1.174,0.0105,1,5tAa8Uaqr4VvA3693mbIhU,5N4erncE7kuUccm7zEmwzk
66,5N4erncE7kuUccm7zEmwzk_4oQQAaU8J9A3JckIuNYCTv,0.3696,0.009,0.071,0.316,26.704,0.000002,7.0,1.0,0.0990,2.345,0.0198,1,5N4erncE7kuUccm7zEmwzk,4oQQAaU8J9A3JckIuNYCTv


In [13]:
df_diff_with_snippets.columns

Index(['current_track-next_track', 'diff_acousticness', 'diff_danceability',
       'diff_energy', 'diff_valence', 'diff_tempo', 'diff_instrumentalness',
       'diff_key', 'diff_mode', 'diff_liveness', 'diff_loudness',
       'diff_speechiness', 'is_good_sequence', 'current_track', 'next_track'],
      dtype='object')

In [14]:
df_diff_with_snippets = df_diff_with_snippets[['current_track', 'next_track','current_track-next_track', 'diff_acousticness', 'diff_danceability',
       'diff_energy', 'diff_valence', 'diff_tempo', 'diff_instrumentalness',
       'diff_key', 'diff_mode', 'diff_liveness', 'diff_loudness',
       'diff_speechiness', 'is_good_sequence']]

In [15]:
# save in .csv can do the 1st time to save time if you use the same information

# df_diff_with_snippets.to_csv(creating_playlist_folder+"df_diff_with_snippets_pop_"+TodaysDate+".csv",index=0)

In [16]:
# load from file 

# df_diff_with_snippets = pd.read_csv(creating_playlist_folder+"df_diff_with_snippets_pop_"+TodaysDate+".csv")

In [17]:
df_diff_with_snippets.head()

,current_track,next_track,current_track-next_track,diff_acousticness,diff_danceability,diff_energy,diff_valence,diff_tempo,diff_instrumentalness,diff_key,diff_mode,diff_liveness,diff_loudness,diff_speechiness,is_good_sequence
62,2nFAQBrEEHA7nvYgw6lpD0,7DHApOHsIFo7EyfieArYGE,2nFAQBrEEHA7nvYgw6lpD0_7DHApOHsIFo7EyfieArYGE,0.3701,0.244,0.066,0.135,90.104,0.000000,10.0,1.0,0.0760,0.306,0.0985,1
63,7DHApOHsIFo7EyfieArYGE,6xUy203RnyyOfbqf96Nven,7DHApOHsIFo7EyfieArYGE_6xUy203RnyyOfbqf96Nven,0.1181,0.245,0.336,0.255,4.895,0.000000,6.0,1.0,0.0530,1.890,0.2375,1
64,6xUy203RnyyOfbqf96Nven,5tAa8Uaqr4VvA3693mbIhU,6xUy203RnyyOfbqf96Nven_5tAa8Uaqr4VvA3693mbIhU,0.2530,0.258,0.090,0.102,31.963,0.000000,4.0,1.0,0.0291,1.036,0.2242,1
65,5tAa8Uaqr4VvA3693mbIhU,5N4erncE7kuUccm7zEmwzk,5tAa8Uaqr4VvA3693mbIhU_5N4erncE7kuUccm7zEmwzk,0.4126,0.077,0.029,0.080,6.737,0.000003,11.0,0.0,0.2411,1.174,0.0105,1
66,5N4erncE7kuUccm7zEmwzk,4oQQAaU8J9A3JckIuNYCTv,5N4erncE7kuUccm7zEmwzk_4oQQAaU8J9A3JckIuNYCTv,0.3696,0.009,0.071,0.316,26.704,0.000002,7.0,1.0,0.0990,2.345,0.0198,1


In [18]:
df_diff_with_snippets.info(null_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 749090 entries, 62 to 6054028
Data columns (total 15 columns):
current_track               749090 non-null object
next_track                  749090 non-null object
current_track-next_track    749090 non-null object
diff_acousticness           749090 non-null float64
diff_danceability           749090 non-null float64
diff_energy                 749090 non-null float64
diff_valence                749090 non-null float64
diff_tempo                  749090 non-null float64
diff_instrumentalness       749090 non-null float64
diff_key                    749090 non-null float64
diff_mode                   749090 non-null float64
diff_liveness               749090 non-null float64
diff_loudness               749090 non-null float64
diff_speechiness            749090 non-null float64
is_good_sequence            749090 non-null int64
dtypes: float64(11), int64(1), object(3)
memory usage: 91.4+ MB


In [19]:
len(df_diff_with_snippets.current_track.unique())

866

# Create initial distance dataframe with audio features information for category pop

Further the Euclidian distance between tracks and also between the midpoint of 2 tracks and the median are calculated after filtering with conditions for each playlist to reduce computational use.

In [20]:
df_distance = df_diff_with_snippets[['current_track-next_track','current_track', 'next_track']]

In [21]:
df_tracks = pd.read_csv(creating_playlist_folder+tracks_file_clean)

In [22]:
df_distance['energy_median'] = df_tracks['energy'].describe()['50%']
df_distance['valence_median'] = df_tracks['valence'].describe()['50%']
df_distance['tempo_median'] = df_tracks['tempo'].describe()['50%']

C:\Users\s178288\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\s178288\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\s178288\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [23]:
df_distance = df_distance.merge(df_tracks[['track_id', 'energy','valence','tempo']], left_on = 'current_track', right_on = 'track_id', how = 'left')

In [24]:
df_distance.drop(columns = ['track_id'],inplace = True)

In [25]:
# ct : current_track

df_distance.rename(columns = {'energy':'energy_ct', 'valence':'valence_ct',
       'tempo':'tempo_ct'},inplace = True)

In [26]:
df_distance = df_distance.merge(df_tracks[['track_id', 'energy','valence','tempo']], left_on = 'next_track', right_on = 'track_id', how = 'left')

In [27]:
df_distance.drop(columns = ['track_id'],inplace = True)

In [28]:
# nt = next_track

df_distance.rename(columns = {'energy':'energy_nt', 'valence':'valence_nt',
       'tempo':'tempo_nt'},inplace = True)

In [29]:
# Calculating coordenates of the middle point of 2 tracks
df_distance['energy_mid_pont'] = (df_distance['energy_ct']+df_distance['energy_nt'])/2
df_distance['valence_mid_pont'] = (df_distance['valence_ct']+df_distance['valence_nt'])/2
df_distance['tempo_mid_pont'] = (df_distance['tempo_ct']+df_distance['tempo_nt'])/2

In [30]:
# merge energy, valence, and tempo from df_diff_with_snippets 

df_distance = df_distance.merge(df_diff_with_snippets[['current_track-next_track', 'diff_energy','diff_valence','diff_tempo']], left_on = 'current_track-next_track', right_on = 'current_track-next_track', how = 'left')


In [31]:
# save in csv

# df_distance.to_csv(creating_playlist_folder+"distance_pop_"+TodaysDate+".csv",index=0)

In [34]:
# checking
# df = pd.read_csv(creating_playlist_folder+"distance_pop_"+TodaysDate+".csv")
# df.head()

In [35]:
# del df

# Calculating Percentailes and Outliers

They are calculated on the values of absolute difference of audio features of tracks that occurs in the category. Only for energy, valence, and tempo that are the features of interesse.


In [36]:
df_diff_in_playlists = pd.read_csv(creating_playlist_folder+diff_in_playlists)

## Percentailes

In [37]:
percentailes = [idx for idx in frange(0.0,1,0.05)]

In [38]:
df_percentailes = df_diff_in_playlists[['diff_energy','diff_valence','diff_tempo']].quantile(percentailes, axis = 0)
df_percentailes

,diff_energy,diff_valence,diff_tempo
0.00,0.00000,0.000000,0.00000
0.05,0.01325,0.019000,1.94700
0.10,0.02500,0.041000,3.99950
0.15,0.03800,0.059000,6.00125
0.20,0.05100,0.079000,8.44700
0.25,0.06500,0.096000,10.89000
0.30,0.07900,0.117000,13.42400
0.35,0.09400,0.140000,15.92250
0.40,0.11100,0.161000,18.43000
0.45,0.12600,0.184000,21.86450


## Outliers

In [39]:
from scipy.stats import iqr

iqr_energy = iqr(df_diff_in_playlists['diff_energy'], axis=0)

out_energy_tracks_diff_in_playlists = df_diff_in_playlists['diff_energy'].describe()['75%']+1.5*iqr_energy
print("Energy: outliers above", out_energy_tracks_diff_in_playlists)

iqr_valence = iqr(df_diff_in_playlists['diff_valence'], axis=0)

out_valence_tracks_diff_in_playlists = df_diff_in_playlists['diff_valence'].describe()['75%']+1.5*iqr_valence
print("Valence: outliers above", out_valence_tracks_diff_in_playlists)

iqr_tempo = iqr(df_diff_in_playlists['diff_tempo'], axis=0)
out_tempo_tracks_diff_in_playlists = df_diff_in_playlists['diff_tempo'].describe()['75%']+1.5*iqr_tempo
print("Tempo: outliers above", out_tempo_tracks_diff_in_playlists)

Energy: outliers above 0.5299999999999998
Valence: outliers above 0.7284999999999999
Tempo: outliers above 96.06375000000003


# Generating csv with distances for Playlists

## Distances to be used by playlist 4 - 'low' energy/valence & 'low' tempo with low percentaille onder 20%

Previous analysis made us choose for low percentaile 20% and for high above 90%.

### Filtering (le, lv, lt)

From the 2-track sequences that have snippets sequences with low absolute difference in energy, valence, and tempo are chosen.

In [40]:
le = round(df_percentailes['diff_energy'][0.20],2)
lv = round(df_percentailes['diff_valence'][0.20],2)
lt = round(df_percentailes['diff_tempo'][0.20],2)
print(le,lv,lt)

0.05 0.08 8.45


In [41]:
df_diff_low_low = df_diff_with_snippets[(df_diff_with_snippets['diff_energy'] <= le) & (df_diff_with_snippets['diff_valence'] <= lv) 
                                        & (df_diff_with_snippets['diff_tempo'] <= lt)][['current_track-next_track','diff_energy','diff_valence','diff_tempo']].sort_values(by=['diff_tempo','diff_valence','diff_energy'])

df_diff_low_low.reset_index(drop=True,inplace=True)
df_diff_low_low.head()

,current_track-next_track,diff_energy,diff_valence,diff_tempo
0,2nFAQBrEEHA7nvYgw6lpD0_5vlTxb7BP5DWzpw8RgnzxR,0.0,0.0,0.0
1,78nplE9X2ZGvQh3TKgvldE_7FlHNJT4TC120CDvFOHzei,0.0,0.0,0.0
2,6xtcFXSo8H9BZN637BMVKS_5Fla3zyOCcIRqrDoKLU0DP,0.0,0.0,0.0
3,5vlTxb7BP5DWzpw8RgnzxR_2nFAQBrEEHA7nvYgw6lpD0,0.0,0.0,0.0
4,39IsH7B5byx8NRlEKlZVg9_1NDxZ7cFAo481dtYWdrUnR,0.0,0.0,0.0


In [42]:
df_diff_low_low.shape

(4530, 4)

In [43]:
df_diff_low_low[['diff_energy','diff_valence','diff_tempo']].max()

diff_energy     0.050
diff_valence    0.080
diff_tempo      8.427
dtype: float64

In [44]:
list_low_low_current_next_track = df_diff_low_low["current_track-next_track"].tolist()

In [45]:
# df_distance only with pairs considering the low low restriction
df_distance_low_low = df_distance[df_distance["current_track-next_track"].isin(list_low_low_current_next_track)]

In [46]:
df_distance_low_low.shape

(4530, 18)

In [47]:
df_distance_low_low.reset_index(drop=True, inplace=True)

In [48]:
df_distance_low_low.head()

,current_track-next_track,current_track,next_track,energy_median,valence_median,tempo_median,energy_ct,valence_ct,tempo_ct,energy_nt,valence_nt,tempo_nt,energy_mid_pont,valence_mid_pont,tempo_mid_pont,diff_energy,diff_valence,diff_tempo
0,5tAa8Uaqr4VvA3693mbIhU_5N4erncE7kuUccm7zEmwzk,5tAa8Uaqr4VvA3693mbIhU,5N4erncE7kuUccm7zEmwzk,0.642,0.487,113.815,0.699,0.585,124.896,0.670,0.505,118.159,0.6845,0.5450,121.5275,0.029,0.080,6.737
1,7AkLPOwjju8taQCJCaIUts_1c3ugaImroUqbpKZX9SxvR,7AkLPOwjju8taQCJCaIUts,1c3ugaImroUqbpKZX9SxvR,0.642,0.487,113.815,0.840,0.539,123.068,0.844,0.584,124.014,0.8420,0.5615,123.5410,0.004,0.045,0.946
2,40c73a0DOalvl8MAKQGo3P_5mN3xSyvCmFy1jkdA98IfS,40c73a0DOalvl8MAKQGo3P,5mN3xSyvCmFy1jkdA98IfS,0.642,0.487,113.815,0.788,0.961,102.079,0.806,0.892,98.008,0.7970,0.9265,100.0435,0.018,0.069,4.071
3,3xXltlaLeerkxFFwOKQigE_02bVUI20o1fsVPBy73xcQv,3xXltlaLeerkxFFwOKQigE,02bVUI20o1fsVPBy73xcQv,0.642,0.487,113.815,0.948,0.904,125.032,0.934,0.966,126.028,0.9410,0.9350,125.5300,0.014,0.062,0.996
4,26SdeD5OY1YzFsDQCNnCIp_5lCdO6rLpi5LIqFWSojJzu,26SdeD5OY1YzFsDQCNnCIp,5lCdO6rLpi5LIqFWSojJzu,0.642,0.487,113.815,0.735,0.369,134.973,0.705,0.446,140.927,0.7200,0.4075,137.9500,0.030,0.077,5.954


In [49]:
# calculate distances
df_distance_low_low['euclidian_dist_midpoint_median'] = 0.00
df_distance_low_low['euclidian_dist_track'] = 0.00

C:\Users\s178288\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\s178288\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [50]:
df_distance_low_low.head()

,current_track-next_track,current_track,next_track,energy_median,valence_median,tempo_median,energy_ct,valence_ct,tempo_ct,energy_nt,valence_nt,tempo_nt,energy_mid_pont,valence_mid_pont,tempo_mid_pont,diff_energy,diff_valence,diff_tempo,euclidian_dist_midpoint_median,euclidian_dist_track
0,5tAa8Uaqr4VvA3693mbIhU_5N4erncE7kuUccm7zEmwzk,5tAa8Uaqr4VvA3693mbIhU,5N4erncE7kuUccm7zEmwzk,0.642,0.487,113.815,0.699,0.585,124.896,0.670,0.505,118.159,0.6845,0.5450,121.5275,0.029,0.080,6.737,0.0,0.0
1,7AkLPOwjju8taQCJCaIUts_1c3ugaImroUqbpKZX9SxvR,7AkLPOwjju8taQCJCaIUts,1c3ugaImroUqbpKZX9SxvR,0.642,0.487,113.815,0.840,0.539,123.068,0.844,0.584,124.014,0.8420,0.5615,123.5410,0.004,0.045,0.946,0.0,0.0
2,40c73a0DOalvl8MAKQGo3P_5mN3xSyvCmFy1jkdA98IfS,40c73a0DOalvl8MAKQGo3P,5mN3xSyvCmFy1jkdA98IfS,0.642,0.487,113.815,0.788,0.961,102.079,0.806,0.892,98.008,0.7970,0.9265,100.0435,0.018,0.069,4.071,0.0,0.0
3,3xXltlaLeerkxFFwOKQigE_02bVUI20o1fsVPBy73xcQv,3xXltlaLeerkxFFwOKQigE,02bVUI20o1fsVPBy73xcQv,0.642,0.487,113.815,0.948,0.904,125.032,0.934,0.966,126.028,0.9410,0.9350,125.5300,0.014,0.062,0.996,0.0,0.0
4,26SdeD5OY1YzFsDQCNnCIp_5lCdO6rLpi5LIqFWSojJzu,26SdeD5OY1YzFsDQCNnCIp,5lCdO6rLpi5LIqFWSojJzu,0.642,0.487,113.815,0.735,0.369,134.973,0.705,0.446,140.927,0.7200,0.4075,137.9500,0.030,0.077,5.954,0.0,0.0


In [51]:
# Insert column with Euclidian distance between midpoint of 2 tracks and the median
from tqdm import tqdm

for idx in tqdm(range(len(df_distance_low_low))):
    a = np.array(df_distance_low_low[['energy_mid_pont', 'valence_mid_pont', 'tempo_mid_pont']].iloc[idx])
    b = np.array(df_distance_low_low[['energy_median', 'valence_median', 'tempo_median']].iloc[idx])
    df_distance_low_low['euclidian_dist_midpoint_median'][idx] = np.linalg.norm(a-b)
    c = np.array(df_distance_low_low[['energy_ct', 'valence_ct', 'tempo_ct']].iloc[idx])
    d = np.array(df_distance_low_low[['energy_nt', 'valence_nt', 'tempo_nt']].iloc[idx])
    df_distance_low_low['euclidian_dist_track'][idx] = np.linalg.norm(c-d)

  0%|                                                                                         | 0/4530 [00:00<?, ?it/s]C:\Users\s178288\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\s178288\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\s178288\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame


In [52]:
df_distance_low_low.head()

,current_track-next_track,current_track,next_track,energy_median,valence_median,tempo_median,energy_ct,valence_ct,tempo_ct,energy_nt,valence_nt,tempo_nt,energy_mid_pont,valence_mid_pont,tempo_mid_pont,diff_energy,diff_valence,diff_tempo,euclidian_dist_midpoint_median,euclidian_dist_track
0,5tAa8Uaqr4VvA3693mbIhU_5N4erncE7kuUccm7zEmwzk,5tAa8Uaqr4VvA3693mbIhU,5N4erncE7kuUccm7zEmwzk,0.642,0.487,113.815,0.699,0.585,124.896,0.670,0.505,118.159,0.6845,0.5450,121.5275,0.029,0.080,6.737,7.712835,6.737537
1,7AkLPOwjju8taQCJCaIUts_1c3ugaImroUqbpKZX9SxvR,7AkLPOwjju8taQCJCaIUts,1c3ugaImroUqbpKZX9SxvR,0.642,0.487,113.815,0.840,0.539,123.068,0.844,0.584,124.014,0.8420,0.5615,123.5410,0.004,0.045,0.946,9.728341,0.947078
2,40c73a0DOalvl8MAKQGo3P_5mN3xSyvCmFy1jkdA98IfS,40c73a0DOalvl8MAKQGo3P,5mN3xSyvCmFy1jkdA98IfS,0.642,0.487,113.815,0.788,0.961,102.079,0.806,0.892,98.008,0.7970,0.9265,100.0435,0.018,0.069,4.071,13.779383,4.071624
3,3xXltlaLeerkxFFwOKQigE_02bVUI20o1fsVPBy73xcQv,3xXltlaLeerkxFFwOKQigE,02bVUI20o1fsVPBy73xcQv,0.642,0.487,113.815,0.948,0.904,125.032,0.934,0.966,126.028,0.9410,0.9350,125.5300,0.014,0.062,0.996,11.727375,0.998026
4,26SdeD5OY1YzFsDQCNnCIp_5lCdO6rLpi5LIqFWSojJzu,26SdeD5OY1YzFsDQCNnCIp,5lCdO6rLpi5LIqFWSojJzu,0.642,0.487,113.815,0.735,0.369,134.973,0.705,0.446,140.927,0.7200,0.4075,137.9500,0.030,0.077,5.954,24.135257,5.954573


In [53]:
df_distance_low_low.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4530 entries, 0 to 4529
Data columns (total 20 columns):
current_track-next_track          4530 non-null object
current_track                     4530 non-null object
next_track                        4530 non-null object
energy_median                     4530 non-null float64
valence_median                    4530 non-null float64
tempo_median                      4530 non-null float64
energy_ct                         4530 non-null float64
valence_ct                        4530 non-null float64
tempo_ct                          4530 non-null float64
energy_nt                         4530 non-null float64
valence_nt                        4530 non-null float64
tempo_nt                          4530 non-null float64
energy_mid_pont                   4530 non-null float64
valence_mid_pont                  4530 non-null float64
tempo_mid_pont                    4530 non-null float64
diff_energy                       4530 non-null float64


In [54]:
# save in csv

df_distance_low_low.to_csv(creating_playlist_folder+"distance_low_low_pop_l20_l90_"+TodaysDate+".csv",index=0)

## Distances to be used by playlist 5 - 'low' energy/valence & 'low' tempo with high percentaille above 90%

Previous analysis made us choose for low percentaile 20% and for high between 20% and 50%

### Filtering (le, lv, ht) and outliers

From the 2-track sequences that have snippets sequences with low absolute difference in energy, valence, and high absolute difference for tempo are chosen.

In [55]:
le = round(df_percentailes['diff_energy'][0.20],2)
lv = round(df_percentailes['diff_valence'][0.20],2)
print(le,lv)

0.05 0.08


In [56]:
ht = round(df_percentailes['diff_tempo'][0.90],2)
print(ht)

67.74


In [57]:
df_diff_low_high = df_diff_with_snippets[(df_diff_with_snippets['diff_energy'] <= le) & (df_diff_with_snippets['diff_valence'] <= lv) 
                                        & (df_diff_with_snippets['diff_tempo'] >= ht) & (df_diff_with_snippets['diff_tempo'] < out_tempo_tracks_diff_in_playlists) ][['current_track-next_track','diff_energy','diff_valence','diff_tempo']].sort_values(by=['diff_tempo','diff_valence','diff_energy'])

df_diff_low_high.reset_index(drop=True,inplace=True)
df_diff_low_high.head()

,current_track-next_track,diff_energy,diff_valence,diff_tempo
0,6guTWjJ5Eohg3N4JI2y1q4_4vmgP10TH0YYqJkzr8yLW0,0.019,0.056,67.768
1,4vmgP10TH0YYqJkzr8yLW0_6guTWjJ5Eohg3N4JI2y1q4,0.019,0.056,67.768
2,2O8gSQ9z52tZSLJnuzjkcG_21i3JjSTPryzlVKNBiWjxE,0.035,0.068,67.776
3,21i3JjSTPryzlVKNBiWjxE_2O8gSQ9z52tZSLJnuzjkcG,0.035,0.068,67.776
4,2RlgNHKcydI9sayD2Df2xp_4NH0G33zApUnRNsSK0CwuN,0.006,0.012,67.779


In [58]:
df_diff_low_high.shape

(1452, 4)

In [59]:
df_diff_low_high[['diff_energy','diff_valence','diff_tempo']].min()

diff_energy      0.000
diff_valence     0.000
diff_tempo      67.768
dtype: float64

In [60]:
df_diff_low_high[['diff_energy','diff_valence','diff_tempo']].max()

diff_energy      0.05
diff_valence     0.08
diff_tempo      96.03
dtype: float64

In [61]:
list_low_high_current_next_track = df_diff_low_high["current_track-next_track"].tolist()

In [62]:
# df_distance only with pairs considering the low low restriction
df_distance_low_high = df_distance[df_distance["current_track-next_track"].isin(list_low_high_current_next_track)]

In [63]:
df_distance_low_high.shape

(1452, 18)

In [64]:
df_distance_low_high.reset_index(drop=True, inplace=True)

In [65]:
df_distance_low_high.head()

,current_track-next_track,current_track,next_track,energy_median,valence_median,tempo_median,energy_ct,valence_ct,tempo_ct,energy_nt,valence_nt,tempo_nt,energy_mid_pont,valence_mid_pont,tempo_mid_pont,diff_energy,diff_valence,diff_tempo
0,7sKDftgGirHWTVFMtJoDoh_5600nq7TiBraBX2jobyJ3l,7sKDftgGirHWTVFMtJoDoh,5600nq7TiBraBX2jobyJ3l,0.642,0.487,113.815,0.796,0.551,97.976,0.821,0.509,188.098,0.8085,0.5300,143.0370,0.025,0.042,90.122
1,2CTN6UuO2dNEVZXR0DJVk2_1uru26I2JKd2mQZt0MDCUe,2CTN6UuO2dNEVZXR0DJVk2,1uru26I2JKd2mQZt0MDCUe,0.642,0.487,113.815,0.528,0.376,82.014,0.531,0.321,164.848,0.5295,0.3485,123.4310,0.003,0.055,82.834
2,2CTN6UuO2dNEVZXR0DJVk2_7pdF27mSDuPWhppnHAmWHa,2CTN6UuO2dNEVZXR0DJVk2,7pdF27mSDuPWhppnHAmWHa,0.642,0.487,113.815,0.528,0.376,82.014,0.525,0.369,160.147,0.5265,0.3725,121.0805,0.003,0.007,78.133
3,2CTN6UuO2dNEVZXR0DJVk2_1roCmBlcpV7LzsmB7pmCyt,2CTN6UuO2dNEVZXR0DJVk2,1roCmBlcpV7LzsmB7pmCyt,0.642,0.487,113.815,0.528,0.376,82.014,0.569,0.440,150.267,0.5485,0.4080,116.1405,0.041,0.064,68.253
4,2CTN6UuO2dNEVZXR0DJVk2_708L4ld9wD2pjoqKwaxkXa,2CTN6UuO2dNEVZXR0DJVk2,708L4ld9wD2pjoqKwaxkXa,0.642,0.487,113.815,0.528,0.376,82.014,0.550,0.319,159.971,0.5390,0.3475,120.9925,0.022,0.057,77.957


In [66]:
# calculate distances
df_distance_low_high['euclidian_dist_midpoint_median'] = 0.00
df_distance_low_high['euclidian_dist_track'] = 0.00

C:\Users\s178288\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\s178288\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [67]:
df_distance_low_high.head()

,current_track-next_track,current_track,next_track,energy_median,valence_median,tempo_median,energy_ct,valence_ct,tempo_ct,energy_nt,valence_nt,tempo_nt,energy_mid_pont,valence_mid_pont,tempo_mid_pont,diff_energy,diff_valence,diff_tempo,euclidian_dist_midpoint_median,euclidian_dist_track
0,7sKDftgGirHWTVFMtJoDoh_5600nq7TiBraBX2jobyJ3l,7sKDftgGirHWTVFMtJoDoh,5600nq7TiBraBX2jobyJ3l,0.642,0.487,113.815,0.796,0.551,97.976,0.821,0.509,188.098,0.8085,0.5300,143.0370,0.025,0.042,90.122,0.0,0.0
1,2CTN6UuO2dNEVZXR0DJVk2_1uru26I2JKd2mQZt0MDCUe,2CTN6UuO2dNEVZXR0DJVk2,1uru26I2JKd2mQZt0MDCUe,0.642,0.487,113.815,0.528,0.376,82.014,0.531,0.321,164.848,0.5295,0.3485,123.4310,0.003,0.055,82.834,0.0,0.0
2,2CTN6UuO2dNEVZXR0DJVk2_7pdF27mSDuPWhppnHAmWHa,2CTN6UuO2dNEVZXR0DJVk2,7pdF27mSDuPWhppnHAmWHa,0.642,0.487,113.815,0.528,0.376,82.014,0.525,0.369,160.147,0.5265,0.3725,121.0805,0.003,0.007,78.133,0.0,0.0
3,2CTN6UuO2dNEVZXR0DJVk2_1roCmBlcpV7LzsmB7pmCyt,2CTN6UuO2dNEVZXR0DJVk2,1roCmBlcpV7LzsmB7pmCyt,0.642,0.487,113.815,0.528,0.376,82.014,0.569,0.440,150.267,0.5485,0.4080,116.1405,0.041,0.064,68.253,0.0,0.0
4,2CTN6UuO2dNEVZXR0DJVk2_708L4ld9wD2pjoqKwaxkXa,2CTN6UuO2dNEVZXR0DJVk2,708L4ld9wD2pjoqKwaxkXa,0.642,0.487,113.815,0.528,0.376,82.014,0.550,0.319,159.971,0.5390,0.3475,120.9925,0.022,0.057,77.957,0.0,0.0


In [68]:
# Insert column with Euclidian distance between midpoint of 2 tracks and the median
from tqdm import tqdm

for idx in tqdm(range(len(df_distance_low_high))):
    a = np.array(df_distance_low_high[['energy_mid_pont', 'valence_mid_pont', 'tempo_mid_pont']].iloc[idx])
    b = np.array(df_distance_low_high[['energy_median', 'valence_median', 'tempo_median']].iloc[idx])
    df_distance_low_high['euclidian_dist_midpoint_median'][idx] = np.linalg.norm(a-b)
    c = np.array(df_distance_low_high[['energy_ct', 'valence_ct', 'tempo_ct']].iloc[idx])
    d = np.array(df_distance_low_high[['energy_nt', 'valence_nt', 'tempo_nt']].iloc[idx])
    df_distance_low_high['euclidian_dist_track'][idx] = np.linalg.norm(c-d)

  0%|                                                                                         | 0/1452 [00:00<?, ?it/s]C:\Users\s178288\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\s178288\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\s178288\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame


In [69]:
df_distance_low_high.head()

,current_track-next_track,current_track,next_track,energy_median,valence_median,tempo_median,energy_ct,valence_ct,tempo_ct,energy_nt,valence_nt,tempo_nt,energy_mid_pont,valence_mid_pont,tempo_mid_pont,diff_energy,diff_valence,diff_tempo,euclidian_dist_midpoint_median,euclidian_dist_track
0,7sKDftgGirHWTVFMtJoDoh_5600nq7TiBraBX2jobyJ3l,7sKDftgGirHWTVFMtJoDoh,5600nq7TiBraBX2jobyJ3l,0.642,0.487,113.815,0.796,0.551,97.976,0.821,0.509,188.098,0.8085,0.5300,143.0370,0.025,0.042,90.122,29.222506,90.122013
1,2CTN6UuO2dNEVZXR0DJVk2_1uru26I2JKd2mQZt0MDCUe,2CTN6UuO2dNEVZXR0DJVk2,1uru26I2JKd2mQZt0MDCUe,0.642,0.487,113.815,0.528,0.376,82.014,0.531,0.321,164.848,0.5295,0.3485,123.4310,0.003,0.055,82.834,9.617655,82.834018
2,2CTN6UuO2dNEVZXR0DJVk2_7pdF27mSDuPWhppnHAmWHa,2CTN6UuO2dNEVZXR0DJVk2,7pdF27mSDuPWhppnHAmWHa,0.642,0.487,113.815,0.528,0.376,82.014,0.525,0.369,160.147,0.5265,0.3725,121.0805,0.003,0.007,78.133,7.267320,78.133000
3,2CTN6UuO2dNEVZXR0DJVk2_1roCmBlcpV7LzsmB7pmCyt,2CTN6UuO2dNEVZXR0DJVk2,1roCmBlcpV7LzsmB7pmCyt,0.642,0.487,113.815,0.528,0.376,82.014,0.569,0.440,150.267,0.5485,0.4080,116.1405,0.041,0.064,68.253,2.328719,68.253042
4,2CTN6UuO2dNEVZXR0DJVk2_708L4ld9wD2pjoqKwaxkXa,2CTN6UuO2dNEVZXR0DJVk2,708L4ld9wD2pjoqKwaxkXa,0.642,0.487,113.815,0.528,0.376,82.014,0.550,0.319,159.971,0.5390,0.3475,120.9925,0.022,0.057,77.957,7.179594,77.957024


In [70]:
df_distance_low_high.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1452 entries, 0 to 1451
Data columns (total 20 columns):
current_track-next_track          1452 non-null object
current_track                     1452 non-null object
next_track                        1452 non-null object
energy_median                     1452 non-null float64
valence_median                    1452 non-null float64
tempo_median                      1452 non-null float64
energy_ct                         1452 non-null float64
valence_ct                        1452 non-null float64
tempo_ct                          1452 non-null float64
energy_nt                         1452 non-null float64
valence_nt                        1452 non-null float64
tempo_nt                          1452 non-null float64
energy_mid_pont                   1452 non-null float64
valence_mid_pont                  1452 non-null float64
tempo_mid_pont                    1452 non-null float64
diff_energy                       1452 non-null float64


In [71]:
# save in csv

df_distance_low_high.to_csv(creating_playlist_folder+"distance_low_high_pop_l20_h90_"+TodaysDate+".csv",index=0)

## Building playlist 3 - 'high' energy/valence & 'low' tempo

### Filtering (he, hv, lt) and outliers

From the 2-track sequences that have snippets sequences with high absolute difference in energy, valence, and low absolute difference for tempo are chosen.

In [72]:
lt = round(df_percentailes['diff_tempo'][0.20],2)
print(lt)

8.45


In [73]:
he = round(df_percentailes['diff_energy'][0.90],2)
hv = round(df_percentailes['diff_valence'][0.90],2)
print(he,hv)

0.37 0.49


In [75]:
df_diff_high_low = df_diff_with_snippets[(df_diff_with_snippets['diff_energy'] >= he) & (df_diff_with_snippets['diff_energy'] < out_energy_tracks_diff_in_playlists)
                                         & (df_diff_with_snippets['diff_valence'] >= hv) & (df_diff_with_snippets['diff_valence'] < out_valence_tracks_diff_in_playlists) 
                                        & (df_diff_with_snippets['diff_tempo'] <= lt)][['current_track-next_track','diff_energy','diff_valence','diff_tempo']].sort_values(by=['diff_tempo','diff_valence','diff_energy'])

df_diff_high_low.reset_index(drop=True,inplace=True)
df_diff_high_low.head()

,current_track-next_track,diff_energy,diff_valence,diff_tempo
0,1gVkQo9V1l9eBRd2BE5BCX_4y1LsJpmMti1PfRQV9AWWe,0.425,0.535,0.000
1,4y1LsJpmMti1PfRQV9AWWe_1gVkQo9V1l9eBRd2BE5BCX,0.425,0.535,0.000
2,4y1LsJpmMti1PfRQV9AWWe_5fNrm4tC6DyhfbkzQIDiDd,0.425,0.535,0.000
3,5fNrm4tC6DyhfbkzQIDiDd_4y1LsJpmMti1PfRQV9AWWe,0.425,0.535,0.000
4,1bhUWB0zJMIKr9yVPrkEuI_6aPi3SH0DVAibe7SNNLv2N,0.438,0.575,0.001


In [76]:
df_diff_high_low.shape

(3460, 4)

In [77]:
df_diff_high_low[['diff_energy','diff_valence','diff_tempo']].min()

diff_energy     0.37
diff_valence    0.49
diff_tempo      0.00
dtype: float64

In [78]:
df_diff_high_low[['diff_energy','diff_valence','diff_tempo']].max()

diff_energy     0.529
diff_valence    0.728
diff_tempo      8.439
dtype: float64

In [79]:
list_high_low_current_next_track = df_diff_high_low["current_track-next_track"].tolist()

In [80]:
# df_distance only with pairs considering the low low restriction
df_distance_high_low = df_distance[df_distance["current_track-next_track"].isin(list_high_low_current_next_track)]

In [81]:
df_distance_high_low.shape

(3460, 18)

In [82]:
df_distance_high_low.reset_index(drop=True, inplace=True)

In [83]:
df_distance_high_low.head()

,current_track-next_track,current_track,next_track,energy_median,valence_median,tempo_median,energy_ct,valence_ct,tempo_ct,energy_nt,valence_nt,tempo_nt,energy_mid_pont,valence_mid_pont,tempo_mid_pont,diff_energy,diff_valence,diff_tempo
0,5QJavZ2sUpPrhyMWItEBJC_1pVz7AJO2UZd5ouOhoIlhS,5QJavZ2sUpPrhyMWItEBJC,1pVz7AJO2UZd5ouOhoIlhS,0.642,0.487,113.815,0.220,0.323,70.165,0.614,0.8360,76.017,0.4170,0.57950,73.0910,0.394,0.5130,5.852
1,544Vnwfg8YSUP6An38yUsT_6RPb92J2rlqC1DK7BWIgpF,544Vnwfg8YSUP6An38yUsT,6RPb92J2rlqC1DK7BWIgpF,0.642,0.487,113.815,0.793,0.847,128.046,0.311,0.2070,132.199,0.5520,0.52700,130.1225,0.482,0.6400,4.153
2,3GZFKiGVYv3SBQ6PLf3JgF_1tU5J8aMqCQe7WtqTkFLQL,3GZFKiGVYv3SBQ6PLf3JgF,1tU5J8aMqCQe7WtqTkFLQL,0.642,0.487,113.815,0.256,0.266,95.288,0.629,0.8440,96.198,0.4425,0.55500,95.7430,0.373,0.5780,0.910
3,2ekn2ttSfGqwhhate0LSR0_3NWCPJllg1v12ElS4eQMKV,2ekn2ttSfGqwhhate0LSR0,3NWCPJllg1v12ElS4eQMKV,0.642,0.487,113.815,0.700,0.608,116.073,0.201,0.0763,109.328,0.4505,0.34215,112.7005,0.499,0.5317,6.745
4,2ekn2ttSfGqwhhate0LSR0_1tT55K6VEyO6XFDxK4lDQe,2ekn2ttSfGqwhhate0LSR0,1tT55K6VEyO6XFDxK4lDQe,0.642,0.487,113.815,0.700,0.608,116.073,0.181,0.0747,111.956,0.4405,0.34135,114.0145,0.519,0.5333,4.117


In [84]:
# calculate distances
df_distance_high_low['euclidian_dist_midpoint_median'] = 0.00
df_distance_high_low['euclidian_dist_track'] = 0.00

C:\Users\s178288\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\s178288\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [85]:
df_distance_high_low.head()

,current_track-next_track,current_track,next_track,energy_median,valence_median,tempo_median,energy_ct,valence_ct,tempo_ct,energy_nt,valence_nt,tempo_nt,energy_mid_pont,valence_mid_pont,tempo_mid_pont,diff_energy,diff_valence,diff_tempo,euclidian_dist_midpoint_median,euclidian_dist_track
0,5QJavZ2sUpPrhyMWItEBJC_1pVz7AJO2UZd5ouOhoIlhS,5QJavZ2sUpPrhyMWItEBJC,1pVz7AJO2UZd5ouOhoIlhS,0.642,0.487,113.815,0.220,0.323,70.165,0.614,0.8360,76.017,0.4170,0.57950,73.0910,0.394,0.5130,5.852,0.0,0.0
1,544Vnwfg8YSUP6An38yUsT_6RPb92J2rlqC1DK7BWIgpF,544Vnwfg8YSUP6An38yUsT,6RPb92J2rlqC1DK7BWIgpF,0.642,0.487,113.815,0.793,0.847,128.046,0.311,0.2070,132.199,0.5520,0.52700,130.1225,0.482,0.6400,4.153,0.0,0.0
2,3GZFKiGVYv3SBQ6PLf3JgF_1tU5J8aMqCQe7WtqTkFLQL,3GZFKiGVYv3SBQ6PLf3JgF,1tU5J8aMqCQe7WtqTkFLQL,0.642,0.487,113.815,0.256,0.266,95.288,0.629,0.8440,96.198,0.4425,0.55500,95.7430,0.373,0.5780,0.910,0.0,0.0
3,2ekn2ttSfGqwhhate0LSR0_3NWCPJllg1v12ElS4eQMKV,2ekn2ttSfGqwhhate0LSR0,3NWCPJllg1v12ElS4eQMKV,0.642,0.487,113.815,0.700,0.608,116.073,0.201,0.0763,109.328,0.4505,0.34215,112.7005,0.499,0.5317,6.745,0.0,0.0
4,2ekn2ttSfGqwhhate0LSR0_1tT55K6VEyO6XFDxK4lDQe,2ekn2ttSfGqwhhate0LSR0,1tT55K6VEyO6XFDxK4lDQe,0.642,0.487,113.815,0.700,0.608,116.073,0.181,0.0747,111.956,0.4405,0.34135,114.0145,0.519,0.5333,4.117,0.0,0.0


In [86]:
# Insert column with Euclidian distance between midpoint of 2 tracks and the median
from tqdm import tqdm

for idx in tqdm(range(len(df_distance_high_low))):
    a = np.array(df_distance_high_low[['energy_mid_pont', 'valence_mid_pont', 'tempo_mid_pont']].iloc[idx])
    b = np.array(df_distance_high_low[['energy_median', 'valence_median', 'tempo_median']].iloc[idx])
    df_distance_high_low['euclidian_dist_midpoint_median'][idx] = np.linalg.norm(a-b)
    c = np.array(df_distance_high_low[['energy_ct', 'valence_ct', 'tempo_ct']].iloc[idx])
    d = np.array(df_distance_high_low[['energy_nt', 'valence_nt', 'tempo_nt']].iloc[idx])
    df_distance_high_low['euclidian_dist_track'][idx] = np.linalg.norm(c-d)

  0%|                                                                                         | 0/3460 [00:00<?, ?it/s]C:\Users\s178288\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\s178288\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\s178288\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame


In [87]:
df_distance_high_low.head()

,current_track-next_track,current_track,next_track,energy_median,valence_median,tempo_median,energy_ct,valence_ct,tempo_ct,energy_nt,valence_nt,tempo_nt,energy_mid_pont,valence_mid_pont,tempo_mid_pont,diff_energy,diff_valence,diff_tempo,euclidian_dist_midpoint_median,euclidian_dist_track
0,5QJavZ2sUpPrhyMWItEBJC_1pVz7AJO2UZd5ouOhoIlhS,5QJavZ2sUpPrhyMWItEBJC,1pVz7AJO2UZd5ouOhoIlhS,0.642,0.487,113.815,0.220,0.323,70.165,0.614,0.8360,76.017,0.4170,0.57950,73.0910,0.394,0.5130,5.852,40.724727,5.887640
1,544Vnwfg8YSUP6An38yUsT_6RPb92J2rlqC1DK7BWIgpF,544Vnwfg8YSUP6An38yUsT,6RPb92J2rlqC1DK7BWIgpF,0.642,0.487,113.815,0.793,0.847,128.046,0.311,0.2070,132.199,0.5520,0.52700,130.1225,0.482,0.6400,4.153,16.307797,4.229578
2,3GZFKiGVYv3SBQ6PLf3JgF_1tU5J8aMqCQe7WtqTkFLQL,3GZFKiGVYv3SBQ6PLf3JgF,1tU5J8aMqCQe7WtqTkFLQL,0.642,0.487,113.815,0.256,0.266,95.288,0.629,0.8440,96.198,0.4425,0.55500,95.7430,0.373,0.5780,0.910,18.073229,1.140751
3,2ekn2ttSfGqwhhate0LSR0_3NWCPJllg1v12ElS4eQMKV,2ekn2ttSfGqwhhate0LSR0,3NWCPJllg1v12ElS4eQMKV,0.642,0.487,113.815,0.700,0.608,116.073,0.201,0.0763,109.328,0.4505,0.34215,112.7005,0.499,0.5317,6.745,1.140072,6.784300
4,2ekn2ttSfGqwhhate0LSR0_1tT55K6VEyO6XFDxK4lDQe,2ekn2ttSfGqwhhate0LSR0,1tT55K6VEyO6XFDxK4lDQe,0.642,0.487,113.815,0.700,0.608,116.073,0.181,0.0747,111.956,0.4405,0.34135,114.0145,0.519,0.5333,4.117,0.318773,4.183714


In [88]:
df_distance_high_low.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3460 entries, 0 to 3459
Data columns (total 20 columns):
current_track-next_track          3460 non-null object
current_track                     3460 non-null object
next_track                        3460 non-null object
energy_median                     3460 non-null float64
valence_median                    3460 non-null float64
tempo_median                      3460 non-null float64
energy_ct                         3460 non-null float64
valence_ct                        3460 non-null float64
tempo_ct                          3460 non-null float64
energy_nt                         3460 non-null float64
valence_nt                        3460 non-null float64
tempo_nt                          3460 non-null float64
energy_mid_pont                   3460 non-null float64
valence_mid_pont                  3460 non-null float64
tempo_mid_pont                    3460 non-null float64
diff_energy                       3460 non-null float64


In [89]:
# save in csv

df_distance_high_low.to_csv(creating_playlist_folder+"distance_high_low_pop_l20_h90_"+TodaysDate+".csv",index=0)